In [10]:
import pandas as pd
import json
from pandas import DataFrame
from sklearn import linear_model
import csv
import argparse
import os
from os import listdir
from os.path import isfile, join
from sklearn.metrics import mean_squared_error
from math import sqrt
import boto3
from sklearn.externals import joblib

#Parameters required for S3 access

s3accessKey=''
s3secretKey=''
s3endpointUrl=''
s3objectStoreLocation=''
trainingdata = ""
outputmodel=''

TRAINING_DATA='data.csv'

#Download training data

session = boto3.Session(aws_access_key_id=s3accessKey, aws_secret_access_key=s3secretKey)
s3 = session.resource('s3', endpoint_url=s3endpointUrl, verify=False)
s3.meta.client.download_file(s3objectStoreLocation, trainingdata, TRAINING_DATA)

# Read the dependent and independent variables

with open(TRAINING_DATA, "rt") as f:
 reader = csv.reader(f)
 i = next(reader)

col = i[:]
y_col = i[-1]
del i[-1]
x_col = i


data = pd.read_csv(TRAINING_DATA)
df = DataFrame(data,columns=col)
X = df[x_col]
Y = df[y_col]

#Train the model

regr = linear_model.LinearRegression()
regr.fit(X, Y)

#Save the model

filename = 'model.pkl'
joblib.dump(regr, filename)

#Copy trained model to S3 backend

s3.meta.client.upload_file(filename, s3objectStoreLocation, outputmodel)